<div align="center">

# 🎵 AICoverGen NextGen

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DDME36/AICoverGen-NextGen/blob/main/AICoverGen_NextGen_Colab.ipynb)

**Create AI Cover songs with any voice**

---

| Feature | Description |
|---------|-------------|
| 🎤 BS-RoFormer | SOTA Vocal Separation (SDR 12.97) |
| 🔇 DeEcho-DeReverb | Remove Echo & Reverb |
| 🎵 RVC v2 | High Quality Voice Conversion |
| 🎛️ FCPE/Hybrid | Smoother Pitch Detection |
| ⚡ Optimized | Fast inference with torch.compile |

---

**How to use:** Run Cell 1 → Run Cell 2 to start WebUI

</div>

In [ ]:
#@title 1️⃣ **Install** (Run once, ~3-4 minutes)

import os
import sys
from IPython.display import clear_output

print("="*60)
print("🎵 AICoverGen NextGen - Optimized Installation")
print("="*60)

# Step 1: Python 3.10 (if needed)
print("\n[1/4] 🐍 Checking Python...")
if sys.version_info.minor == 12:
    !sudo apt-get update -qq && sudo apt-get install -y -qq python3.10 python3.10-distutils python3.10-dev
    !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.12 1
    !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 2
    !curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10
print("✓ Python ready")

# Step 2: Clone
print("\n[2/4] 📂 Cloning repository...")
REPO = '/content/AICoverGen-NextGen'
if os.path.exists(REPO):
    !cd {REPO} && git pull -q
else:
    !git clone -q --depth 1 https://github.com/DDME36/AICoverGen-NextGen.git {REPO}
os.chdir(REPO)
print("✓ Repository ready")

# Step 3: Dependencies (optimized - parallel install)
print("\n[3/4] 📦 Installing dependencies...")
!sudo apt-get install -y -qq build-essential python3-dev sox libsox-fmt-all 2>/dev/null

# Uninstall old torch first
!pip uninstall -y torch torchaudio torchvision -q 2>/dev/null

# Fix pip version for fairseq compatibility
!pip install -q "pip<24.1"

# Install PyTorch with CUDA
!pip install -q torch==2.5.1 torchaudio==2.5.1 torchvision==0.20.1 --index-url https://download.pytorch.org/whl/cu121

# Install dependencies
!pip install -q gradio==3.50.2 numpy==1.26.4 scipy==1.13.1
!pip install -q librosa==0.10.2 soundfile pydub pedalboard pyworld praat-parselmouth
!pip install -q onnxruntime-gpu faiss-cpu torchcrepe torchfcpe
!pip install -q fairseq==0.12.2
!pip install -q "audio-separator[gpu]"
!pip install -q yt-dlp sox ffmpeg-python
!pip install -q "numpy<2"

print("✓ Dependencies installed")

# Step 4: Models
print("\n[4/4] 🤖 Downloading AI models...")
!python setup_models.py

# Enable torch optimizations
import torch
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

clear_output()
print("="*60)
print("✅ Installation Complete!")
print("="*60)
print()
print("📦 Installed:")
print("   • PyTorch 2.5.1 + CUDA 12.1 (optimized)")
print("   • BS-RoFormer (Vocal Separation)")
print("   • FCPE/Hybrid Pitch Detection")
print("   • RVC v2 + Auto-Mix")
print()
print("⚡ Optimizations enabled:")
print("   • cuDNN benchmark mode")
print("   • TF32 acceleration")
print()
print("👉 Now run Cell 2 to start WebUI!")
print("="*60)

In [ ]:
#@title 2️⃣ **Run WebUI** 🚀

import os
import gc
import torch

os.chdir('/content/AICoverGen-NextGen')

# Clear GPU memory
gc.collect()
torch.cuda.empty_cache()

# Enable optimizations
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# Fix np.int deprecation
with open('src/vc_infer_pipeline.py', 'r') as f:
    content = f.read().replace('astype(np.int)', 'astype(int)')
with open('src/vc_infer_pipeline.py', 'w') as f:
    f.write(content)

# Fix matplotlib backend
for key in ['MPLBACKEND', 'DISPLAY']:
    if key in os.environ:
        del os.environ[key]
os.environ['MPLBACKEND'] = 'Agg'

# Show GPU info
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"🖥️  GPU: {gpu_name} ({gpu_mem:.1f} GB)")

print("="*60)
print("🎵 AICoverGen NextGen")
print("="*60)
print()
print("⏳ Starting WebUI...")
print("🔗 Click the https://xxxxx.gradio.live link below")
print()
print("📝 Quick Guide:")
print("   1. Download voice model in 'Download model' tab")
print("   2. Go to 'Generate' tab")
print("   3. Paste YouTube link or upload file")
print("   4. Select voice model & click Generate")
print()
print("🎚️ Pitch Detection:")
print("   • rmvpe: Fast, clear vocals")
print("   • fcpe: Smoother, less robotic")
print("   • hybrid: Best quality (recommended)")
print()
print("⚠️  Keep this cell running!")
print("="*60 + "\n")

!python src/webui.py --share